In [28]:
#Import the necessary libraries
import cv2
import numpy as np
import time
import os
from pathlib import Path

In [29]:
#Import Directory for OpenCV screenshots
screenshot_dir = 'CV_Screenshots'
os.makedirs(screenshot_dir, exist_ok=True)

In [30]:
#PHASE 1: CAMERA ACCESS AND SETUP

def find_working_camera():
    print("🔍 Searching for working camera...\n")
    
    # Try with AVFoundation (Mac-specific)
    cap = cv2.VideoCapture(1, cv2.CAP_AVFOUNDATION)
        
    if cap.isOpened():
            # Try to read a frame
        ret, frame = cap.read()
        if ret and frame is not None:
            h, w = frame.shape[:2]
            print(f"✅ WORKS! ({w}x{h})")
            cap.release()
            return 1
        else:
            print("❌ Opens but can't grab frames")
    else:
        print("❌ Can't open")
        
    cap.release()
    
    return None

In [31]:

def test_camera_access():
    # Find working camera
    camera_index = find_working_camera()
    
    if camera_index is None:
        print("\n❌ ERROR: No working camera found!")
        print("💡 Troubleshooting:")
        print("   1. Close any apps using camera (Zoom, Teams, FaceTime)")
        print("   2. System Settings > Privacy & Security > Camera")
        print("      → Enable for Terminal/Python/VS Code")
        print("   3. Restart your Python kernel/terminal")
        return
    
    print(f"\n✅ Using camera {camera_index}")
    
    # Open camera with AVFoundation backend (better for Mac)
    cap = cv2.VideoCapture(camera_index, cv2.CAP_AVFOUNDATION)
    
    # Force better settings
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    cap.set(cv2.CAP_PROP_FPS, 30)
    
    print("✅ Camera opened successfully!")
    print("\n📋 Instructions:")
    print("   - Press 'q' to quit")
    print("   - Press 's' to save screenshot")
    print("   - ESC also quits")
    print("\nCamera window should open now...\n")
    
    # Get camera properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    print(f"📊 Camera Info:")
    print(f"   Resolution: {width}x{height}")
    print(f"   FPS: {fps}")
    
    screenshot_count = 0
    frame_count = 0
    start_time = time.time()
    
    # Main camera loop
    while True:
        # Read frame from camera
        ret, frame = cap.read()
        
        if not ret:
            print("❌ Failed to grab frame")
            break
        
        frame_count += 1
        
        # Calculate actual FPS
        elapsed_time = time.time() - start_time
        if elapsed_time > 0:
            actual_fps = frame_count / elapsed_time
        else:
            actual_fps = 0
        
        # Add FPS counter to frame
        cv2.putText(
            frame, 
            f"FPS: {actual_fps:.1f}", 
            (10, 30),
            cv2.FONT_HERSHEY_SIMPLEX, 
            0.7, 
            (0, 255, 0), 
            2
        )
        
        # Add instructions
        cv2.putText(
            frame, 
            "Press 'q' to quit, 's' for screenshot", 
            (10, height - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 
            0.5, 
            (255, 255, 255), 
            1
        )
        
        # Display the frame
        cv2.imshow('Friend Classifier - Phase 1', frame)
        
        # Handle keyboard input
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord('q') or key == 27:  # 'q' or ESC
            print("\n👋 Quitting...")
            break
        elif key == ord('s'):  # Save screenshot
            screenshot_count += 1
            filename = os.path.join(screenshot_dir, f'screenshot_{screenshot_count}.jpg')
            cv2.imwrite(filename, frame)
            print(f"📸 Screenshot saved: {filename}")
    
    # Cleanup
    cap.release()
    cv2.destroyAllWindows()
    
    print("\n" + "="*60)
    print("✅ PHASE 1 COMPLETE!")
    print(f"   Total frames: {frame_count}")
    print(f"   Average FPS: {actual_fps:.1f}")
    print(f"   Screenshots: {screenshot_count}")
    print("="*60)


In [32]:
# ============================================
# PHASE 2: FACE DETECTION
# ============================================

def load_face_detector():
    """
    Load the Haar Cascade face detector.
    
    Returns:
        face_cascade: OpenCV cascade classifier for face detection
    """
    
    print("📦 Loading Haar Cascade face detector...")
    
    cascade_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
    face_cascade = cv2.CascadeClassifier(cascade_path)
    
    if face_cascade.empty():
        print("❌ ERROR: Could not load face detector!")
        print(f"   Tried path: {cascade_path}")
        return None
    
    print("✅ Face detector loaded successfully!")
    return face_cascade


def detect_faces(frame, face_cascade, scale_factor=1.1, min_neighbors=5, min_size=(30, 30)):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=scale_factor,
        minNeighbors=min_neighbors,
        minSize=min_size,
        flags=cv2.CASCADE_SCALE_IMAGE
    )
    return faces

In [33]:
# ============================================
# PHASE 3: FACE PREPROCESSING
# ============================================

def create_output_folder(folder_name="CV_PrePrep_Faces"):
    folder_path = Path(folder_name)
    folder_path.mkdir(exist_ok=True)
    
    print(f"📁 Output folder ready: {folder_path.absolute()}")
    
    return folder_path


def crop_face(frame, face_coords, margin=20): 
    x, y, w, h = face_coords
    frame_height, frame_width = frame.shape[:2]
    
    # Add margin but stay within frame bounds
    x1 = max(0, x - margin)
    y1 = max(0, y - margin)
    x2 = min(frame_width, x + w + margin)
    y2 = min(frame_height, y + h + margin)
    
    # Crop the face
    face_crop = frame[y1:y2, x1:x2]
    
    actual_coords = (x1, y1, x2-x1, y2-y1)
    
    return face_crop, actual_coords


def preprocess_face_for_model(face_crop, target_size=(128, 128), normalize=True):
    """
    Preprocess a cropped face for model prediction.
    This MUST match your training preprocessing!
    
    Steps:
        1. Convert BGR (OpenCV) to RGB (model format)
        2. Resize to model's expected input size
        3. Normalize pixel values to 0-1 range
        4. Add batch dimension
    
    Args:
        face_crop: Cropped face image (BGR from OpenCV)
        target_size: Size your model expects (default: 128x128)
        normalize: Whether to normalize to 0-1 range (default: True)
    
    Returns:
        processed_face: Face ready for model.predict()
        display_face: Face for visualization (RGB, not normalized)
    """
    
    # Step 1: Convert BGR to RGB (OpenCV uses BGR, model expects RGB)
    face_rgb = cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB)
    
    # Step 2: Resize to model's expected input size
    face_resized = cv2.resize(face_rgb, target_size, interpolation=cv2.INTER_AREA)
    
    # Step 3: Create display version (before normalization)
    display_face = face_resized.copy()
    
    # Step 4: Normalize pixel values to 0-1 range (if needed)
    if normalize:
        face_normalized = face_resized.astype(np.float32) / 255.0
    else:
        face_normalized = face_resized.astype(np.float32)
    
    # Step 5: Add batch dimension (model expects: batch_size, height, width, channels)
    face_batched = np.expand_dims(face_normalized, axis=0)
    
    return face_batched, display_face


def save_preprocessed_face(face_image, folder_path, prefix="face", timestamp=None):
    """
    Save a preprocessed face to disk.
    
    Args:
        face_image: Face image to save (RGB or BGR)
        folder_path: Path object to save folder
        prefix: Filename prefix
        timestamp: Optional timestamp for unique filename
    
    Returns:
        filename: Path to saved file
    """
    
    if timestamp is None:
        timestamp = time.strftime("%Y%m%d_%H%M%S")
    
    # Generate unique filename
    filename = folder_path / f"{prefix}_{timestamp}.jpg"
    
    # Convert RGB to BGR for saving (OpenCV saves in BGR)
    if face_image.dtype == np.float32:
        # If normalized, denormalize first
        face_image = (face_image * 255).astype(np.uint8)
    
    face_bgr = cv2.cvtColor(face_image, cv2.COLOR_RGB2BGR)
    
    # Save
    cv2.imwrite(str(filename), face_bgr)
    
    return filename


def visualize_preprocessing_pipeline(face_crop, target_size=(128, 128)):
    """
    Show each preprocessing step side-by-side.
    Useful for understanding what happens to the image.
    
    Args:
        face_crop: Original cropped face
        target_size: Target size for model
    
    Returns:
        combined: Visualization showing all 4 steps
    """
    
    # Original (BGR)
    step1 = face_crop
    
    # Convert to RGB
    step2 = cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB)
    
    # Resize
    step3 = cv2.resize(step2, target_size)
    
    # Normalize (for visualization, convert back to 0-255)
    step4_normalized = step3.astype(np.float32) / 255.0
    step4 = (step4_normalized * 255).astype(np.uint8)
    
    # Create visualization
    h, w = 200, 200
    
    # Resize all to same size for display
    display1 = cv2.resize(step1, (w, h))
    display2 = cv2.resize(cv2.cvtColor(step2, cv2.COLOR_RGB2BGR), (w, h))
    display3 = cv2.resize(cv2.cvtColor(step3, cv2.COLOR_RGB2BGR), (w, h))
    display4 = cv2.resize(cv2.cvtColor(step4, cv2.COLOR_RGB2BGR), (w, h))
    
    # Add labels
    def add_label(img, text):
        labeled = img.copy()
        cv2.putText(labeled, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 
                   0.7, (0, 255, 0), 2)
        return labeled
    
    display1 = add_label(display1, "1. Original")
    display2 = add_label(display2, "2. BGR->RGB")
    display3 = add_label(display3, f"3. Resize {target_size}")
    display4 = add_label(display4, "4. Normalized")
    
    # Combine horizontally
    top_row = np.hstack([display1, display2])
    bottom_row = np.hstack([display3, display4])
    combined = np.vstack([top_row, bottom_row])
    
    return combined


# ============================================
# MAIN DEMO FUNCTION
# ============================================

def face_preprocessing_demo(camera_index=1):
    """
    Interactive demo showing face preprocessing in real-time.
    
    Controls:
        - Press 'q' to quit
        - Press 's' to save preprocessed face
        - Press 'v' to show preprocessing steps
        - Press 'd' to toggle debug info
    """
    
    print("\n" + "="*60)
    print("🔪 PHASE 2+3: FACE DETECTION & PREPROCESSING")
    print("="*60)
    
    # Setup
    face_cascade = load_face_detector()
    if face_cascade is None:
        return
    
    output_folder = create_output_folder("CV_PrePrep_Faces")
    
    cap = cv2.VideoCapture(camera_index, cv2.CAP_AVFOUNDATION)
    if not cap.isOpened():
        print("❌ Failed to open camera")
        return
    
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    
    print("\n📋 Controls:")
    print("   q - Quit")
    print("   s - Save preprocessed face")
    print("   v - Visualize preprocessing steps")
    print("   d - Toggle debug info")
    print("\nWindow opening...\n")
    
    # State
    show_debug = True
    saved_count = 0
    frame_count = 0
    start_time = time.time()
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        
        # Detect faces
        faces = detect_faces(frame, face_cascade)
        
        # Process each detected face
        for i, (x, y, w, h) in enumerate(faces):
            # Draw detection box
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            
            # Crop face
            face_crop, _ = crop_face(frame, (x, y, w, h), margin=20)
            
            if face_crop.size == 0:
                continue
            
            # Preprocess for model
            face_processed, face_display = preprocess_face_for_model(
                face_crop, 
                target_size=(128, 128)
            )
            
            # Display preprocessed face in corner
            corner_size = 150
            face_corner = cv2.resize(
                cv2.cvtColor(face_display, cv2.COLOR_RGB2BGR),
                (corner_size, corner_size)
            )
            
            # Position in top-right corner
            frame[10:10+corner_size, frame.shape[1]-corner_size-10:frame.shape[1]-10] = face_corner
            
            # Add border around preview
            cv2.rectangle(
                frame,
                (frame.shape[1]-corner_size-10, 10),
                (frame.shape[1]-10, 10+corner_size),
                (255, 255, 0),
                2
            )
            
            # Show preprocessing info
            if show_debug:
                info_x = x
                info_y = y - 10
                
                cv2.putText(
                    frame,
                    f"Face {i+1}",
                    (info_x, info_y),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6,
                    (0, 255, 0),
                    2
                )
                
                # Show dimensions
                debug_text = [
                    f"Original: {face_crop.shape[1]}x{face_crop.shape[0]}",
                    f"Processed: 128x128",
                    f"Shape: {face_processed.shape}",
                    f"Range: [0.0, 1.0]"
                ]
                
                for j, text in enumerate(debug_text):
                    cv2.putText(
                        frame,
                        text,
                        (info_x, y + h + 20 + j*20),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.4,
                        (255, 255, 255),
                        1
                    )
        
        # Add overlay info
        fps = frame_count / (time.time() - start_time)
        
        cv2.putText(frame, f"FPS: {fps:.1f}", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        cv2.putText(frame, f"Faces: {len(faces)}", (10, 60),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        cv2.putText(frame, f"Saved: {saved_count}", (10, 90),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        cv2.putText(frame, "Preview (preprocessed)", 
                   (frame.shape[1]-240, frame.shape[0]-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)
        
        cv2.putText(frame, "q: Quit | s: Save | v: Visualize | d: Debug",
                   (10, frame.shape[0]-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        # Display
        cv2.imshow('Phase 2+3: Detection & Preprocessing', frame)
        
        # Handle input
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord('q'):
            break
        
        elif key == ord('s') and len(faces) > 0:
            # Save first detected face
            face_crop, _ = crop_face(frame, faces[0], margin=20)
            _, face_display = preprocess_face_for_model(face_crop)
            
            timestamp = time.strftime("%Y%m%d_%H%M%S_%f")[:-3]
            filename = save_preprocessed_face(face_display, output_folder, timestamp=timestamp)
            
            saved_count += 1
            print(f"💾 Saved: {filename}")
        
        elif key == ord('v') and len(faces) > 0:
            # Show preprocessing visualization
            face_crop, _ = crop_face(frame, faces[0], margin=20)
            vis = visualize_preprocessing_pipeline(face_crop)
            
            cv2.imshow('Preprocessing Steps', vis)
            print("📊 Preprocessing visualization opened (close window to continue)")
        
        elif key == ord('d'):
            show_debug = not show_debug
            print(f"🐛 Debug info: {'ON' if show_debug else 'OFF'}")
    
    # Cleanup
    cap.release()
    cv2.destroyAllWindows()
    
    print("\n" + "="*60)
    print("✅ PHASE 2+3 COMPLETE!")
    print(f"   Frames processed: {frame_count}")
    print(f"   Faces saved: {saved_count}")
    print(f"   Output folder: {output_folder.absolute()}")
    print("="*60)


# ============================================
# USAGE
# ============================================

if __name__ == "__main__":
    
    print("""
    ╔══════════════════════════════════════════════════════════╗
    ║                                                          ║
    ║         FRIEND FACE CLASSIFIER - CAMERA SYSTEM          ║
    ║            Phase 2+3: Detection & Preprocessing          ║
    ║                                                          ║
    ╚══════════════════════════════════════════════════════════╝
    """)
    
    # Run the demo
    face_preprocessing_demo(camera_index=1)
    
    print("\n🎉 Phase 2+3 Complete!")
    print("\nWhat you learned:")
    print("  ✅ How to detect faces with Haar Cascades")
    print("  ✅ How to crop faces from frames")
    print("  ✅ How to resize to model's expected size")
    print("  ✅ How to normalize pixel values (0-1 range)")
    print("  ✅ How to convert BGR → RGB")
    print("  ✅ How to add batch dimension")
    print("\nNext steps:")
    print("  ✅ Phase 1: Camera access")
    print("  ✅ Phase 2: Face detection")
    print("  ✅ Phase 3: Face preprocessing (DONE!)")
    print("  ⏭️  Phase 4: Load model & classify")
    print("  ⏭️  Phase 5: Final integration & polish")


    ╔══════════════════════════════════════════════════════════╗
    ║                                                          ║
    ║         FRIEND FACE CLASSIFIER - CAMERA SYSTEM          ║
    ║            Phase 2+3: Detection & Preprocessing          ║
    ║                                                          ║
    ╚══════════════════════════════════════════════════════════╝
    

🔪 PHASE 2+3: FACE DETECTION & PREPROCESSING
📦 Loading Haar Cascade face detector...
✅ Face detector loaded successfully!
📁 Output folder ready: /Users/enzobasuil/Desktop/Coding Projects/Python Projects/FriendIdentifier/CV_PrePrep_Faces

📋 Controls:
   q - Quit
   s - Save preprocessed face
   v - Visualize preprocessing steps
   d - Toggle debug info

Window opening...

📊 Preprocessing visualization opened (close window to continue)
📊 Preprocessing visualization opened (close window to continue)
📊 Preprocessing visualization opened (close window to continue)
📊 Preprocessing visualization opened (c